In [12]:
R_DATE = 20220831
#P_DATE1 = 20220731
#P_DATE2 = 20220630
#P_DATE3 = 20220531
#P_DATE4 = 20220430
#P_DATE5 = 20220331

#Added 23/6/2022 for GIL From Impaired
#year = 2022        #current year
#year1 = 202204     #current month

import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan


#dalam data source ---> business financing
BB = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\01_Portfolio\\'+str(R_DATE)+'\\Master BB_'+str(R_DATE)+'.xlsx', sheet_name='Master Raw')
BB_RDMS = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\12_RDMS\\'+str(R_DATE)+'\\RDMS_BB_'+str(R_DATE)+'.xlsx')
#BB_NA = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\03_Newly_Approved\\'+str(R_DATE)+'\\Loan Approval_'+str(R_DATE)+'.xlsx')
#BB_EWS = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\02_Asset_Quality\\'+str(R_DATE)+'\\Asset_Quality_Master List_'+str(R_DATE)+'.xlsx', sheet_name='EWS')
#BB_WATCHLIST = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\02_Asset_Quality\\'+str(R_DATE)+'\\Asset_Quality_Master List_'+str(R_DATE)+'.xlsx', sheet_name='Watchlist')
#BB_SMA = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\02_Asset_Quality\\'+str(R_DATE)+'\\Asset_Quality_Master List_'+str(R_DATE)+'.xlsx', sheet_name='SMA')
#BB_RnR = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\02_Asset_Quality\\'+str(R_DATE)+'\\Asset_Quality_Master List_'+str(R_DATE)+'.xlsx', sheet_name='R&R')



#MIB_BB_CUS_PDATE1 = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\01_Portfolio\\'+str(P_DATE1)+'\\BB2_SUMMARY_'+str(P_DATE1)+'.xlsx', sheet_name='CUSBASE2')
#MIB_BB_CUS_PDATE2 = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\01_Portfolio\\'+str(P_DATE2)+'\\BB2_SUMMARY_'+str(P_DATE2)+'.xlsx', sheet_name='CUSBASE2')
#MIB_BB_CUS_PDATE3 = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\01_Portfolio\\'+str(P_DATE3)+'\\BB2_SUMMARY_'+str(P_DATE3)+'.xlsx', sheet_name='CUSBASE2')
#MIB_BB_CUS_PDATE4 = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\01_Portfolio\\'+str(P_DATE4)+'\\BB2_SUMMARY_'+str(P_DATE4)+'.xlsx', sheet_name='CUSBASE2')
#MIB_BB_CUS_PDATE5 = pd.read_excel (r'T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\02_Business_Financing\01_Portfolio\\'+str(P_DATE5)+'\\BB2_SUMMARY_'+str(P_DATE5)+'.xlsx', sheet_name='CUSBASE2')

Total_Impaired = pd.read_excel (r"T:\MIB Risk Management\Credit Risk Analytics - Historical\02_Data_Source\20_CRP\6.0 Impaired Loan\\"+str(R_DATE)[:4]+"\\"+str(R_DATE)[:6]+"\working\(MIB) Total_impaired_"+str(R_DATE)[:6]+".xlsx")
Total_Impaired.columns = Total_Impaired.columns.str.replace(" ", "_")

BB.columns = BB.columns.str.replace(" ", "_")
BB_RDMS.columns = BB_RDMS.columns.str.replace(" ", "_")
#BB_NA.columns = BB_NA.columns.str.replace(" ", "_")
#BB_EWS.columns = BB_EWS.columns.str.replace(" ", "_")
#BB_WATCHLIST.columns = BB_WATCHLIST.columns.str.replace(" ", "_")
#BB_SMA.columns = BB_SMA.columns.str.replace(" ", "_")
#BB_RnR.columns = BB_RnR.columns.str.replace(" ", "_")

In [13]:
def RISKCD(x):
    if x >= 1 and x<= 5:
        return '01.Very Low Risk'
    elif x >= 6 and x <= 10:
        return '02.Low Risk'
    elif x >= 11 and x <= 15:
        return '03.Moderate Risk'
    elif x >= 16 and x <= 21:
        return '04.High Risk'
    elif x >= 22 and x <= 23:
        return '05.Sub Standard'
    else:
        return '06.Unrated'

def FRRCD(x):
    if x == 'A+':
        return '01.Secured'
    elif x == 'A':
        return '01.Secured'
    elif x == 'B':
        return '02.Partially Secured'
    elif x == 'C':
        return '02.Partially Secured'
    elif x == 'D':
        return '02.Partially Secured'
    elif x == 'E':
        return '02.Partially Secured'
    elif x == 'F':
        return '02.Partially Secured'
    elif x == 'G':
        return '03.Clean'
    else:
        return '03.Clean'
    
BB1 = BB.iloc[np.where((BB.Facility_Level_2!='Non-Funded Tradebills')&(BB.Facility_Level_2!='FEC (10%)'))]

BB1['ISC_NOB'] = BB1['ISC_NOB'].map(str)
BB1['ISC_NOB'] = BB1['ISC_NOB'].str.strip()
BB1['ISC_NOB'] = BB1['ISC_NOB'].str.upper()
BB1.loc[BB1.ISC_NOB.isin(['01120','0113X','01140','0119XA','0119XB','01261A','01261B','01262A','01262B','01263','01269','01273','01279','01291','01299','0130X','0141X','0144X','0145X','0146XA','0146XB','0146XC','0149X','01500','016XX','0170X','0210X','0220X','0230X','0240X','031XXA','031XXB','031XXC','031XXD','032XXA','032XXB','032XXC','1120','113X','1140','119XA','119XB','1261A','1261B','1262A','1262B','1263','1269','1273','1279','1291','1299','130X','141X','144X','145X','146XA','146XB','146XC','149X','1500','16XX','170X','210X','220X','230X','240X','31XXA','31XXB','31XXC','31XXD','32XXA','32XXB','32XXC']),'ISC_PARENT1'] = "01. Agriculture, Forestry and Fishing"
BB1.loc[BB1.ISC_NOB.isin(['05XXX','0610X','0620X','0710X','07210','07291','07299','08XXX','0910XA','0910XB','0910XC','0910XD','0910XE','0910XF','0910XG','09900', '5XXX','610X','620X','710X','7210','7291','7299','8XXX','910XA','910XB','910XC','910XD','910XE','910XF','910XG','9900']),'ISC_PARENT1'] = "02. Mining and Quarrying"
BB1.loc[BB1.ISC_NOB.isin(['1010X','1020X','1030X','10401A','10401B','10402A','10402B','10499','1050X','106XX','107XX','10800','110XXA','110XXB','12000','13XXX','14XXX','151XX','1520X','16XXX','1701X','181XX','18200','19XXXA','19XXXB','20XXX','21XXX','22XXXA','22XXXB','22XXXC','22XXXD','22XXXE','22XXXF','22XXXG','23XXXA','23XXXB','23XXXC','2410XA','2410XB','24201','24202','24209A','24209B','2431XA','2431XB','24320','251XX','25200','25910','25920','25930','2599X','26300','265XX','2670X','26999','27500','27999','28170','2819XA','2819XB','2819XC','2819XD','2819XE','28220','28299A','28299B','28299C','28299D','28299E','29XXXA','29XXXB','29XXXC','301XXA','301XXB','301XXC','301XXD','30200','30999A','30999B','30999C','3100X','32500','3290X','33XXXA','33XXXB','33XXXC','33XXXD','33XXXE','33XXXF','33XXXG','33XXXH']),'ISC_PARENT1'] =  "03. Manufacturing";
BB1.loc[BB1.ISC_NOB.isin(['3510XA','3510XB','3510XC','3510XD','3510XE','3510XF','3510XG','3520XA','3520XB','3530X']),'ISC_PARENT1'] = "04. Electricity, Gas, Steam and Air Conditioning Supply"
BB1.loc[BB1.ISC_NOB.isin(['360XX','3700X','38112','38199','382XXA','382XXB','382XXC','3830X','39000']),'ISC_PARENT1']="05. Water Supply; Sewerage, Waste Management and Remediation Activities"
BB1.loc[BB1.ISC_NOB.isin(['41001A','41001B','41001C','41001D','41001E','41001F','41001G','41001H','41002A','41002B','41002C','41002D','41002E','41002F','41002G','41002H','41002I','41002J','41003','41009','4210XA','4210XB','4210XC','4210XD','4210XE','4210XF','4210XG','4220XA','4220XB','4220XC','4220XD','429XXA','429XXB','429XXC','431XX','432XX','4330X','4390X']),'ISC_PARENT1'] = "06. Construction"
BB1.loc[BB1.ISC_NOB.isin(['451XXA','451XXB','451XXC','451XXD','451XXE','4520X','4530XA','4530XB','4540X','462XXA','462XXB','462XXC','463XXA','463XXB','463XXC','463XXD','4641X','46496','46499','46510','46595','46599A','46599B','46599C','46999','4711X','4719X','472XXA','472XXB','472XXC','472XXD','478XX','479XX','40799A','40799B','40799C','40799D','40799E']),'ISC_PARENT1'] = "07. Wholesale and Retail Trade; Repair of Motor Vehicles and Motorcycles"
BB1.loc[BB1.ISC_NOB.isin(['4911X','4912X','4921X','4922X','49230','49300','5011X','50121','50122','5021X','5022X','51101A','51101B','51102','51103','51201','51202','51203','5210XA','5210XB','5221X','5222X','5223X','52241','52249','52291','52292','52299','53XXX']),'ISC_PARENT1'] = "08. Transportation and Storage"
BB1.loc[BB1.ISC_NOB.isin(['55101A','55101B','55101C','55101D','55104','55105','55108','55199','5520X','55900','56101','56102','56103','56104','56105','56106','56107','562XX','56302','56303','56399A','56399B']),'ISC_PARENT1'] = "09. Accommodation and Food Service Activities"
BB1.loc[BB1.ISC_NOB.isin(['58110','5812X','5813X','5819X','582XX','59XXX','60XXX','6110X','6120X','6130X','6190X','620XX','631XX','6391X','63990']),'ISC_PARENT1'] = "10. Information and Communication"
BB1.loc[BB1.ISC_NOB.isin(['64110','64191','64192','64193','64194','64195','64199','64200','64301','64302','64303','64304','64309','64910','64921','64922','64923','64924','64925','64929','64991','64992','64993','64999','651XX','6520X','65301','65302','66111','66112','66113','66114','66119','66121','66122','66123','66124','66125','66129','66191','66192','66199','66211','66212','66221','66222','66223','66224','66290','66301','66302','66303']),'ISC_PARENT1'] = "11. Financial and Insurance/ Takaful Activities"
BB1.loc[BB1.ISC_NOB.isin(['68101A','68101B','68101C','68102A','68102B','68102C','68102D','68102E','68103A','68103B','68104A','68104B','68104C','68104D','68104E','68104F','68104G','68109','68201','68202','68203','68209']),'ISC_PARENT1'] =  "12. Real Estate Activities"
BB1.loc[BB1.ISC_NOB.isin(['69XXX','70100','70201','70202','70203','70209','7110X','71200','72102','72103','72105','72106','72199','7220X','73100','73200','74XXX','75000']),'ISC_PARENT1'] =  "13. Professional, Scientific and Technical Activities"
BB1.loc[BB1.ISC_NOB.isin(['7710X','772XX','77301','77307','77399','77400','78XXX','79XXX','80XXX','81XXX','821XX','82200','8230X','82910','8292XA','8292XB','8292XC','82990']),'ISC_PARENT1'] = "14. Administrative and Support Service Activities"
BB1.loc[BB1.ISC_NOB.isin(['841XX','842XX','8430X']),'ISC_PARENT1'] = "15. Public Administration and Defence; Compulsory Social Security"
BB1.loc[BB1.ISC_NOB.isin(['8510X','852XX','8530X','854XX','8550X']),'ISC_PARENT1'] = "16. Education"
BB1.loc[BB1.ISC_NOB.isin(['861XX','8620X','8690X','87XXX','88XXX']),'ISC_PARENT1'] = "17. Human Health and Social Work Activities"
BB1.loc[BB1.ISC_NOB.isin(['9000X','910XX','92000','93XXX']),'ISC_PARENT1'] = "18. Arts, Entertainment and Recreation"
BB1.loc[BB1.ISC_NOB.isin(['94XXXA','94XXXB','95XXX','960XX']),'ISC_PARENT1'] = "19. Other Service Activities"
BB1.loc[BB1.ISC_NOB.isin(['TXXXX']),'ISC_PARENT1'] =  "20. Activities of Households as Employers; Undifferentiated Goods and Services Producing Activities of Households for Own Use"
BB1.loc[BB1.ISC_NOB.isin(['99000']),'ISC_PARENT1'] = "21. Activities of Extraterritorial Organisations and Bodies"
BB1.loc[BB1.ISC_NOB.isin(['97000']),'ISC_PARENT1'] = "22. Household"
BB1.ISC_PARENT1.fillna(-9999, inplace=True)
BB1.loc[BB1.ISC_NOB.isin([-9999]),'ISC_PARENT1'] = "23. Not Populated"


#--------------------------------------------------BB_ACCT------------------------------------------------

BB_ACCT1 = BB1.drop('ISC_PARENT', axis = 1)

BB_ACCT1 = BB_ACCT1.rename(columns = {'ISC_PARENT1':'ISC_PARENT'})
BB_ACCT1.loc[BB_ACCT1['BRR'] == 'Unrated', "BRR"] = -9999
BB_ACCT1['BRR'] = BB_ACCT1['BRR'].map(int)

BB_ACCT1['NOA'] = 1
BB_ACCT1['OSM'] = BB_ACCT1['outstanding_balance']/1000000

Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER = Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER.astype(str)
Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER = Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER.str.upper()
Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER = Total_Impaired.V_ORIGINAL_ACCOUNT_NUMBER.str.strip()

BB_ACCT1.Account_Num = BB_ACCT1.Account_Num.astype(str)
BB_ACCT1.Account_Num = BB_ACCT1.Account_Num.str.upper()
BB_ACCT1.Account_Num = BB_ACCT1.Account_Num.str.strip()

#usual case
BB_ACCT = BB_ACCT1.merge(Total_Impaired[['V_ORIGINAL_ACCOUNT_NUMBER','GIL']].rename(columns={'V_ORIGINAL_ACCOUNT_NUMBER': 'Account_Num'}), on = 'Account_Num', how = 'left')

#casefor may2022
#BB_ACCT = BB_ACCT1.merge(Total_Impaired[['V_ORIGINAL_ACCOUNT_NUMBER','GIL']], on = 'V_ORIGINAL_ACCOUNT_NUMBER', how = 'left')

BB_ACCT.GIL.fillna(0, inplace=True)

BB_ACCT['GIF'] = BB_ACCT.GIL

BB_ACCT.drop(['GIL'], axis = 1, inplace=True)


BB_ACCT['PL_IPL_NPL'] = BB_ACCT['PL_IPL_NPL'].str.strip()
BB_ACCT['PL_IPL_NPL'] = BB_ACCT['PL_IPL_NPL'].str.upper()

BB_ACCT.loc[BB_ACCT.PL_IPL_NPL.isin(["IPL","NPL","IPL R&R"]),'NOA_GIL'] = 1
BB_ACCT.loc[~(BB_ACCT.PL_IPL_NPL.isin(["IPL","NPL","IPL R&R"])),'NOA_GIL'] = 0

BB_ACCT.loc[BB_ACCT.PL_IPL_NPL.isin(["IPL","NPL","IPL R&R"]),'OSM_GIL'] = BB_ACCT['GIF']
BB_ACCT.loc[~(BB_ACCT.PL_IPL_NPL.isin(["IPL","NPL","IPL R&R"])),'OSM_GIL'] = 0

#BB_ACCT.loc[BB_ACCT.PL_IPL_NPL.isin(["IPL","NPL","IPL R&R"]),'FINAL_RATING'] = 23
#BB_ACCT.loc[~(BB_ACCT.PL_IPL_NPL.isin(["IPL","NPL","IPL R&R"])),'FINAL_RATING'] = BB_ACCT['BRR']

BB_ACCT['FINAL_RATING'] = BB_ACCT['BRR']

BB_ACCT['FINAL_RATING'] = BB_ACCT['FINAL_RATING'].map(int)

BB_ACCT['RISK_CAT'] = BB_ACCT.FINAL_RATING.apply(RISKCD)
BB_ACCT.loc[BB_ACCT.PL_IPL_NPL.isin(["IPL","NPL","IPL R&R"]),'RISK_CAT'] = '05.Sub Standard'

BB_ACCT['COLLATERAL_TYPE'] = BB_ACCT.FRR.apply(FRRCD)

BB_ACCT.rename(columns={'Risk_Cat': 'Risk_Cat1'}, inplace=True)

#--------------------------------------------------BB_RDMS-------------------------------------------

BB_RDMS['GCIF'] =  BB_RDMS.COUNTERPARTY_ORIGINAL*1 

RDMS_001 = BB_RDMS.sort_values('GCIF', ascending=True).fillna(0)

RDMS_CUS_ALL1 = RDMS_001.groupby(['DEALBOOK','GCIF'])[['EAD','RWA','EL']].sum().reset_index().rename(columns={'DEALBOOK':'Organizational_Unit','EAD':'CUS_EAD','RWA':'CUS_RWA','EL':'CUS_EL'})

RDMS_CUS_ALL = RDMS_CUS_ALL1.iloc[np.where(RDMS_CUS_ALL1.Organizational_Unit.isin({"MIB_OTH","MIB_RA"}))]

C001 = BB_ACCT.merge(RDMS_CUS_ALL[['GCIF','CUS_EAD','CUS_RWA','CUS_EL']],on='GCIF',how='left')

C002_1 = C001#.merge(BB_ACCT[['GCIF','PL_IPL_NPL']].drop_duplicates('GCIF'),on='GCIF',how='left')

C002_1['RISK_CAT'] = C002_1.FINAL_RATING.apply(RISKCD)
C002_1.loc[C002_1.PL_IPL_NPL.isin(["IPL","NPL","IPL R&R"]),'RISK_CAT'] = '05.Sub Standard'
C002 = C002_1.drop('PL_IPL_NPL', axis = 1).drop_duplicates()

#C002['NOC'] = 1
C002['GIL_OSM'] = C002['OSM_GIL']/1000000
C002['EAD_OSM'] = C002['CUS_EAD']/1000000
C002['RWA_OSM'] = C002['CUS_RWA']/1000000
C002['EL_OSM'] = C002['CUS_EL']/1000000

#C002['CUS_OSM'] = C002['outstanding_balance']/1000000


#C002['TOP_RANK'] = C002['outstanding_balance'].rank(ascending=False)


C003 = C002[['REPORT_DATE','Account_Num','GCIF','Borrower_Name','NOA','OSM','NOA_GIL','OSM_GIL','FINAL_RATING',\
             'RISK_CAT','COLLATERAL_TYPE','CUS_EAD','CUS_RWA','CUS_EL','ISC_PARENT','Account_Status','Customer_Class',\
             'BORW_RISK_RATG_BEF_OVRD','RISK_RATG_DT','Ageing','FRR','BRR','NPL_Indicator','REGION','BC','ISC_DESC',\
            'Broad_Sector','Subsector','Funded_non_funded','Facility_Level_2','MBB_Islamic_Ind']].\
rename(columns={'Account_Num':'Account_No','':'','Subsector':'Sub_Sector','Funded_non_funded':'Funded_non_Funded'})

C003.loc[C003.REGION=='FEDERAL TERRITORY', 'REGION'] = 'KUALA LUMPUR'

In [16]:
C003.to_excel(r"S:\Power BI\BB\BB_portfolio"+str(R_DATE)+".xlsx", index = False)


# Append BB

In [17]:
DATE1 = 20220831
DATE2 = 20220731

import numpy as np
import pandas as pd
import pandasql
import sqlite3

pd.set_option("display.max_columns", None) 
pd.set_option("display.max_colwidth", 1000) #huruf dlm column
pd.set_option("display.max_rows", 100)
pd.set_option("display.precision", 2) #2 titik perpuluhan


BB_latest = pd.read_excel (r"S:\Power BI\BB\BB_portfolio"+str(DATE1)+".xlsx", index = False)
BB_previous = pd.read_excel (r"S:\Power BI\BB\BB_portfolio"+str(DATE2)+".xlsx", index = False)

appendR = pd.concat([ BB_latest, BB_previous])

appendR.to_excel(r"S:\Power BI\BB\BB_Combine_"+str(DATE2)+"-"+str(DATE1)+".xlsx", index = False)
